# Watermarking System - Main Notebook

This notebook provides a complete workflow for:
1. Setting up the environment
2. Generating watermarked datasets
3. Generating tampered/attacked datasets
4. Training ML-based watermarking models (Phase 3)
5. Running verification and robustness tests
6. Generating evaluation reports

## Setup Instructions

### For Google Colab Users:
1. **Mount Google Drive** - The setup cell will do this automatically
2. **Set BASE_DIR** - Already configured for `/content/drive/MyDrive/project_codes/models_new`
3. **Upload your project** - Make sure your project files are in the BASE_DIR on Google Drive

### Folder Structure:
- `dataset/originals/` - Place your original images here (should be ready and can be empty initially)
- `dataset/watermarked/` - Will be created automatically (can be empty or will be populated)
- `dataset/tampered/` - Will be created automatically (can be empty or will be populated)
- `dataset/metadata/` - Will be created automatically
- `outputs/` - Will be created automatically for checkpoints, logs, and reports

**Yes, you should have these folders ready (they can be empty) - the code will populate them!**

### Important:
- **Run cells in order** - Especially Step 1 (Setup) before any other steps
- **Check BASE_DIR** - Make sure it points to your project root in Step 1


## ⚠️ IMPORTANT: NumPy Compatibility Fix

**Dependency Warnings Are Expected:**
- You'll see warnings about numpy version conflicts (e.g., opencv wants numpy>=2.0, but we need numpy 1.x for scipy)
- **These warnings are OK** - the code will still work despite the warnings
- OpenCV 4.8.1 works fine with numpy 1.26.4, even though pip shows warnings

**If you get numpy/scipy import errors:**
1. Run Cell 4 (Install Dependencies) below
2. **RESTART THE RUNTIME** (Runtime → Restart runtime)
3. Run Cell 4 again
4. Continue with the rest

**Why this happens:**
- numpy 2.x is incompatible with scipy 1.11.x
- Some packages (opencv 4.12+, pytensor, jax) require numpy 2.x
- We force numpy 1.x for compatibility with scipy
- OpenCV 4.8.1 works with numpy 1.x despite pip warnings


In [ ]:
# QUICK FIX: Run this if you're getting numpy/scipy import errors
# This will force reinstall compatible versions

print("Force fixing numpy/scipy compatibility...")
!pip uninstall -y numpy scipy
!pip install --no-cache-dir "numpy==1.26.4" "scipy==1.11.4"

# Test import
try:
    import numpy as np
    import scipy
    print(f"✓ numpy {np.__version__} - OK")
    print(f"✓ scipy {scipy.__version__} - OK")
    print("\n✓ Compatibility fixed! Now RESTART RUNTIME and continue.")
except Exception as e:
    print(f"⚠ Error: {e}")
    print("Please restart runtime and try again")


## Step 1: Install Dependencies and Setup


In [ ]:
# Install required packages
print("Installing required packages...")
print("Fixing numpy/scipy compatibility (this is critical!)...")
print("⚠ This may take a few minutes and show dependency warnings - that's OK\n")

# STEP 1: Force uninstall numpy 2.x and install compatible numpy 1.x
print("Step 1: Installing compatible numpy version...")
# Uninstall all numpy versions
!pip uninstall -y numpy numpy-base || true
# Also uninstall packages that force numpy 2.x (optional - will show errors if not installed)
!pip uninstall -y pytensor shap jax jaxlib opencv-python-headless || true
# Install numpy 1.26.4 (last stable 1.x version)
!pip install -q "numpy==1.26.4" --no-cache-dir --force-reinstall

# STEP 2: Install scipy (compatible with numpy 1.x)
print("\nStep 2: Installing scipy...")
!pip install -q "scipy==1.11.4" --no-cache-dir

# STEP 3: Install opencv with compatible numpy version (ignore dependency warnings)
print("\nStep 3: Installing OpenCV (will show warnings - that's OK)...")
# Install older opencv version (4.8.1) that works with numpy 1.x
# Using --no-deps to avoid numpy version conflicts
# Note: Dependency warnings are OK - opencv will work with numpy 1.x despite the warnings
import subprocess
import sys

def install_opencv(package_name, version):
    """Install opencv package ignoring dependency conflicts."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", f"{package_name}=={version}", "--no-deps"])
        print(f"  ✓ Installed {package_name}")
    except:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", f"{package_name}=={version}"])
            print(f"  ✓ Installed {package_name} (with dependencies)")
        except:
            print(f"  ⚠ {package_name} installation had issues but may still work")

install_opencv("opencv-python", "4.8.1.78")
install_opencv("opencv-contrib-python", "4.8.1.78")

# STEP 4: Install other core libraries
print("\nStep 4: Installing core libraries...")
!pip install -q "pillow==11.0.0"
!pip install -q pywavelets
!pip install -q reedsolo
!pip install -q cryptography

# STEP 5: Install deep learning libraries
print("\nStep 5: Installing deep learning libraries...")
# Install PyTorch - will auto-detect CUDA if available
!pip install -q torch torchvision torchaudio

# STEP 6: Install utility libraries
print("\nStep 6: Installing utility libraries...")
!pip install -q tqdm
!pip install -q pyyaml
!pip install -q lpips
!pip install -q scikit-image

print("\n✓ All packages installed!")

# Verify numpy version
try:
    import numpy as np
    np_version = np.__version__
    major_version = int(np_version.split('.')[0])
    print(f"\n✓ Installed numpy version: {np_version}")
    
    if major_version >= 2:
        print("⚠ WARNING: numpy 2.x detected! This will cause scipy errors.")
        print("  Trying to force reinstall numpy 1.26.4...")
        !pip uninstall -y numpy numpy-base
        !pip install -q "numpy==1.26.4" --force-reinstall --no-cache-dir
        # Reload numpy
        import importlib
        importlib.reload(sys.modules.get('numpy', None))
        import numpy as np
        np_version = np.__version__
        major_version = int(np_version.split('.')[0])
        print(f"  After fix: numpy {np_version}")
    
    if major_version < 2:
        print("✓ numpy version is compatible (< 2.0)")
    else:
        print("⚠ WARNING: Still numpy 2.x - RESTART RUNTIME and run this cell again")
        
except Exception as e:
    print(f"⚠ Error checking numpy: {e}")
    print("  Please RESTART RUNTIME and run this cell again")

print("\n✓ Package installation complete!")
print("\n" + "="*60)
print("CRITICAL: RESTART THE RUNTIME NOW!")
print("Runtime > Restart runtime")
print("Then run Cell 4 (Install Dependencies) again")
print("="*60)


In [ ]:
# Setup paths and directories
import os
import sys
from pathlib import Path

# ============================================
# GOOGLE COLAB SETUP
# ============================================
# Mount Google Drive (uncomment if not already mounted)
try:
    from google.colab import drive
    # Check if already mounted
    if os.path.exists('/content/drive/MyDrive'):
        print("✓ Google Drive already mounted")
    else:
        drive.mount('/content/drive', force_remount=False)
        print("✓ Google Drive mounted")
except Exception as e:
    print(f"⚠ Google Drive mount issue: {e}")
    print("If you get I/O errors, try remounting:")
    print("  from google.colab import drive")
    print("  drive.flush_and_unmount()")
    print("  drive.mount('/content/drive')")

# Set your project path here for Google Colab
BASE_DIR = Path('/content/drive/MyDrive/project_codes/models_new')

# ============================================
# LOCAL EXECUTION (if not using Colab)
# ============================================
# Uncomment and modify if running locally:
# BASE_DIR = Path.cwd()
# # Or specify your local path:
# # BASE_DIR = Path(r"C:\Users\rehem\Downloads\other models\others\project2")

# Verify the path exists
if not BASE_DIR.exists():
    print(f"⚠ Warning: BASE_DIR does not exist: {BASE_DIR}")
    print("  Trying to find project root...")
    # Try to find project root by looking for key files
    possible_paths = [
        Path('/content/drive/MyDrive/project_codes/models_new'),
        Path('/content/drive/MyDrive/project2'),
        Path.cwd(),
        Path.cwd().parent,
    ]
    for path in possible_paths:
        if path.exists() and (path / "models").exists() and (path / "training").exists():
            BASE_DIR = path
            print(f"  Found project at: {BASE_DIR}")
            break
    else:
        print("  Could not find project root automatically")
        print(f"  Current working directory: {Path.cwd()}")
        print(f"  Please set BASE_DIR manually in this cell")

# Change to project directory (important for Colab)
os.chdir(BASE_DIR)
print(f"✓ Changed working directory to: {BASE_DIR}")

# Add project root to path (at the beginning to ensure it's found first)
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

# Verify the path is correct
print(f"\nWorking directory: {BASE_DIR}")
print(f"Python path includes: {BASE_DIR}")

# Verify key directories exist
if (BASE_DIR / "models").exists() and (BASE_DIR / "training").exists():
    print("✓ Project structure verified")
    print(f"  - models/ directory: ✓")
    print(f"  - training/ directory: ✓")
else:
    print("⚠ Warning: Could not find 'models' or 'training' directories")
    print(f"  Current directory: {BASE_DIR}")
    print(f"  Contents: {list(BASE_DIR.iterdir())}")


In [ ]:
# Create necessary directories
directories = [
    BASE_DIR / "dataset" / "originals",
    BASE_DIR / "dataset" / "watermarked",
    BASE_DIR / "dataset" / "tampered",
    BASE_DIR / "dataset" / "metadata",
    BASE_DIR / "outputs" / "checkpoints",
    BASE_DIR / "outputs" / "logs",
    BASE_DIR / "outputs" / "eval_reports",
    BASE_DIR / "outputs" / "visual_results",
]

for directory in directories:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created/verified: {directory}")

print("\n✓ All directories ready!")

# Verify imports work
print("\nVerifying imports and versions...")

# Check numpy/scipy versions first
try:
    import numpy as np
    import scipy
    print(f"✓ numpy {np.__version__} imported")
    print(f"✓ scipy {scipy.__version__} imported")
    
    # Check if numpy version is compatible (should be < 2.0)
    np_version = tuple(map(int, np.__version__.split('.')[:2]))
    if np_version[0] >= 2:
        print(f"⚠ WARNING: numpy {np.__version__} may cause compatibility issues")
        print("  Consider restarting runtime and reinstalling with: numpy<2.0")
except ImportError as e:
    print(f"⚠ numpy/scipy import error: {e}")

try:
    # Test pywavelets import (the correct way)
    import pywt
    print("✓ pywavelets (pywt) imported successfully")
except ImportError as e:
    print(f"⚠ pywavelets import error: {e}")
    print("  Try restarting the runtime and running the installation cell again")

try:
    import training.utils_data
    import models.hybrid_multidomain_embed
    print("✓ Project modules imported successfully")
except ImportError as e:
    print(f"⚠ Import error: {e}")
    print(f"  Make sure you're running from the project root directory")
    print(f"  Current BASE_DIR: {BASE_DIR}")
    print(f"  sys.path: {sys.path[:3]}")  # Show first 3 paths
    print("\n  If you see numpy/scipy errors, restart runtime and run installation cell again")


## Step 2: Check Dataset


In [ ]:
# Check if original images exist
# Make sure BASE_DIR is in path (in case this cell is run independently)
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

try:
    from training.utils_data import DatasetGenerator
except ImportError as e:
    print(f"Error importing: {e}")
    print(f"BASE_DIR: {BASE_DIR}")
    print(f"Checking if training directory exists: {(BASE_DIR / 'training').exists()}")
    print(f"Contents of BASE_DIR: {list(BASE_DIR.iterdir())}")
    raise

originals_dir = BASE_DIR / "dataset" / "originals"

# First, check if directory exists and list ALL files
print(f"Checking directory: {originals_dir}")
print(f"Directory exists: {originals_dir.exists()}")

# Try multiple methods to access files (Google Drive can be finicky)
all_files = []
image_files = []
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.JPG', '.JPEG', '.PNG', '.BMP', '.TIF', '.TIFF']

if originals_dir.exists():
    try:
        # Method 1: Try pathlib iterdir
        all_files = list(originals_dir.iterdir())
        print(f"\n✓ Successfully accessed directory using pathlib")
    except (OSError, IOError) as e:
        print(f"\n⚠ Pathlib access failed: {e}")
        print("Trying alternative method (os.listdir)...")
        
        try:
            # Method 2: Try os.listdir
            import os
            file_names = os.listdir(str(originals_dir))
            all_files = [originals_dir / name for name in file_names]
            print(f"✓ Successfully accessed directory using os.listdir")
        except (OSError, IOError) as e2:
            print(f"⚠ os.listdir also failed: {e2}")
            print("\n🔧 Google Drive sync issue detected!")
            print("\nSolutions:")
            print("  1. Unmount and remount Google Drive:")
            print("     - Run: drive.flush_and_unmount()")
            print("     - Then remount with: drive.mount('/content/drive')")
            print("  2. Check if files are actually in Google Drive")
            print("  3. Try accessing via Colab file browser (left sidebar)")
            print(f"  4. Verify path exists: {originals_dir}")
            
            # Try to create a test to see if we can write
            try:
                test_file = originals_dir / ".test_write"
                test_file.touch()
                test_file.unlink()
                print("  ✓ Directory is writable")
            except Exception as e3:
                print(f"  ⚠ Directory may not be accessible: {e3}")
            
            # Set empty list so we can continue
            all_files = []
    
    # Process files if we got any
    if len(all_files) > 0:
        print(f"\nTotal files/folders in directory: {len(all_files)}")
        
        print("\nAll files/folders found:")
        for f in all_files[:20]:  # Show first 20
            try:
                file_type = "DIR" if f.is_dir() else "FILE"
                print(f"  [{file_type}] {f.name} ({f.suffix if f.suffix else 'no extension'})")
            except:
                print(f"  [?] {f.name if hasattr(f, 'name') else str(f)}")
        if len(all_files) > 20:
            print(f"  ... and {len(all_files) - 20} more")
        
        # Check for images with various extensions (case-insensitive)
        for file in all_files:
            try:
                if file.is_file():
                    ext = file.suffix
                    if ext in image_extensions or ext.lower() in [e.lower() for e in image_extensions]:
                        image_files.append(file)
            except:
                # Skip files that can't be accessed
                continue
        
        print(f"\n{'='*60}")
        print(f"Found {len(image_files)} images")
        print(f"{'='*60}")
        
        if len(image_files) > 0:
            print(f"\nImage files found:")
            for img in image_files[:10]:
                print(f"  ✓ {img.name} ({img.suffix})")
            if len(image_files) > 10:
                print(f"  ... and {len(image_files) - 10} more images")
            print("\n✓ Ready to proceed with watermarking!")
        else:
            print("\n⚠ No image files found with standard extensions")
            print("\nTroubleshooting:")
            print("  1. Check if images are in subdirectories:")
            subdirs = [f for f in all_files if f.is_dir()]
            if subdirs:
                for subdir in subdirs[:5]:
                    try:
                        subdir_files = list(subdir.iterdir())
                        print(f"     - {subdir.name}/ has {len(subdir_files)} items")
                    except:
                        print(f"     - {subdir.name}/ (cannot access)")
            print("  2. Check file extensions - supported: .jpg, .jpeg, .png, .bmp, .tif, .tiff")
            print("  3. Make sure files are uploaded to Google Drive at:")
            print(f"     {originals_dir}")
            print("  4. Try refreshing the file browser in Colab")
    else:
        print("\n⚠ No files found in directory")
        print("This could mean:")
        print("  1. Directory is empty")
        print("  2. Google Drive sync issue (try remounting)")
        print("  3. Files are in a different location")
else:
    print(f"\n⚠ Directory does not exist: {originals_dir}")
    print("Creating directory...")
    try:
        originals_dir.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created: {originals_dir}")
        print("\nPlease upload your images to this directory and run this cell again.")
    except Exception as e:
        print(f"⚠ Could not create directory: {e}")
        print("You may need to create it manually in Google Drive")


## Step 3.5: Split Dataset into Train/Validation/Test Sets

**Important:** This step splits your images into training (70%), validation (20%), and testing (10%) sets to avoid data leakage and overfitting.


In [ ]:
# Split dataset into Train/Validation/Test sets
import shutil
import random
from pathlib import Path

# Configuration
TOTAL_IMAGES_TO_USE = 18000  # Set to None to use all images, or specify a number
TRAIN_PERCENT = 0.70
VAL_PERCENT = 0.20
TEST_PERCENT = 0.10

# Verify percentages sum to 1.0
assert abs(TRAIN_PERCENT + VAL_PERCENT + TEST_PERCENT - 1.0) < 0.001, "Percentages must sum to 1.0"

# Source and destination directories
originals_dir = BASE_DIR / "dataset" / "originals"
train_dir = BASE_DIR / "dataset" / "train"
val_dir = BASE_DIR / "dataset" / "val"
test_dir = BASE_DIR / "dataset" / "test"

# Create split directories
for split_dir in [train_dir, val_dir, test_dir]:
    split_dir.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created/verified: {split_dir}")

# Get all image files
print(f"\nCollecting images from: {originals_dir}")
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.JPG', '.JPEG', '.PNG', '.BMP', '.TIF', '.TIFF']

all_images = []
for ext in image_extensions:
    all_images.extend(list(originals_dir.glob(f'*{ext}')))

# Remove duplicates and sort for reproducibility
all_images = sorted(list(set(all_images)))
total_images = len(all_images)

print(f"Found {total_images} total images")

# Limit to specified number if requested
if TOTAL_IMAGES_TO_USE and TOTAL_IMAGES_TO_USE < total_images:
    print(f"Limiting to {TOTAL_IMAGES_TO_USE} images (random selection)")
    random.seed(42)  # For reproducibility
    all_images = random.sample(all_images, TOTAL_IMAGES_TO_USE)
    total_images = len(all_images)

# Calculate split sizes
train_size = int(total_images * TRAIN_PERCENT)
val_size = int(total_images * VAL_PERCENT)
test_size = total_images - train_size - val_size  # Remaining goes to test

print(f"\nDataset Split Configuration:")
print(f"  Total images: {total_images}")
print(f"  Training: {train_size} images ({TRAIN_PERCENT*100:.1f}%)")
print(f"  Validation: {val_size} images ({VAL_PERCENT*100:.1f}%)")
print(f"  Testing: {test_size} images ({TEST_PERCENT*100:.1f}%)")

# Shuffle images with fixed seed for reproducibility
random.seed(42)
shuffled_images = all_images.copy()
random.shuffle(shuffled_images)

# Split images
train_images = shuffled_images[:train_size]
val_images = shuffled_images[train_size:train_size + val_size]
test_images = shuffled_images[train_size + val_size:]

print(f"\nSplitting images...")

# Copy images to respective directories
def copy_images(image_list, dest_dir, split_name):
    """Copy images to destination directory."""
    for img_path in image_list:
        dest_path = dest_dir / img_path.name
        try:
            shutil.copy2(img_path, dest_path)
        except Exception as e:
            print(f"  ⚠ Error copying {img_path.name}: {e}")
    print(f"  ✓ {split_name}: {len(image_list)} images copied to {dest_dir}")

copy_images(train_images, train_dir, "Training")
copy_images(val_images, val_dir, "Validation")
copy_images(test_images, test_dir, "Testing")

# Save split information for reproducibility
split_info = {
    'total_images': total_images,
    'train_size': train_size,
    'val_size': val_size,
    'test_size': test_size,
    'train_percent': TRAIN_PERCENT,
    'val_percent': VAL_PERCENT,
    'test_percent': TEST_PERCENT,
    'random_seed': 42,
    'train_images': [str(img.name) for img in train_images],
    'val_images': [str(img.name) for img in val_images],
    'test_images': [str(img.name) for img in test_images]
}

split_info_path = BASE_DIR / "dataset" / "split_info.json"
import json
with open(split_info_path, 'w') as f:
    json.dump(split_info, f, indent=2)

print(f"\n✓ Split information saved to: {split_info_path}")
print(f"\n{'='*60}")
print("Dataset Split Complete!")
print(f"{'='*60}")
print(f"Training set: {train_dir} ({train_size} images)")
print(f"Validation set: {val_dir} ({val_size} images)")
print(f"Testing set: {test_dir} ({test_size} images)")
print(f"\nNote: Original images remain in {originals_dir}")
print("The splits are reproducible (seed=42) and saved in split_info.json")


## Step 3: Initialize Watermarking Models


In [ ]:
# Import watermarking models
from models.hybrid_multidomain_embed import HybridMultiDomainEmbedder
from models.hybrid_multidomain_verify import HybridMultiDomainVerifier
from models.hybrid_multidomain_embed_det import HybridMultiDomainEmbedderDet
from models.hybrid_multidomain_verify_det import HybridMultiDomainVerifierDet

# PATCH: Fix the bytearray conversion issue - COMPREHENSIVE FIX
# This fixes the "sequence item 0: expected a bytes-like object, int found" error
import types
import struct
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def _patched_create_deterministic_payload(self, encoded_payload):
    """Patched version - handles bytearray conversion correctly."""
    # Convert encoded_payload to bytes for RSA signing
    # encoded_payload is bytearray from prepare_payload
    if isinstance(encoded_payload, bytearray):
        payload_bytes = bytes(encoded_payload)  # This conversion always works
    elif isinstance(encoded_payload, bytes):
        payload_bytes = encoded_payload
    else:
        # Convert iterable to bytes via list comprehension
        payload_bytes = bytes([int(b) & 0xFF for b in encoded_payload])
    
    # Sign the payload
    signature = self.private_key.sign(
        payload_bytes,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    # Create headers
    payload_len = len(encoded_payload)
    sig_len = len(signature)
    header = struct.pack('>I', payload_len)  # Returns bytes
    sig_header = struct.pack('>I', sig_len)  # Returns bytes
    
    # Concatenate: convert all to bytearray for compatibility
    # header and sig_header are bytes, signature is bytes, encoded_payload is bytearray
    # CRITICAL: Don't wrap bytearray in bytearray() - that causes the error!
    header_ba = bytearray(header)  # bytes -> bytearray
    payload_ba = encoded_payload if isinstance(encoded_payload, bytearray) else bytearray(encoded_payload)
    sig_header_ba = bytearray(sig_header)  # bytes -> bytearray
    signature_ba = bytearray(signature)  # bytes -> bytearray
    
    final_payload = header_ba + payload_ba + sig_header_ba + signature_ba
    
    return final_payload, signature

def _patched_prepare_payload(self, message: str):
    """Patched version - handles reedsolo output correctly."""
    from typing import Tuple, Dict
    
    # Convert message to bytes
    message_bytes = message.encode('utf-8')
    
    # Encode with Reed-Solomon ECC
    # reedsolo.encode() returns bytearray - use it directly
    rs_encoded = self.rs_codec.encode(message_bytes)
    
    # CRITICAL FIX: Use bytearray directly, don't wrap it
    # The error happens when you do bytearray(bytearray(...))
    if isinstance(rs_encoded, bytearray):
        encoded_payload = rs_encoded  # Use directly - don't wrap!
    elif isinstance(rs_encoded, bytes):
        encoded_payload = bytearray(rs_encoded)  # Convert bytes to bytearray
    else:
        # Fallback: build bytearray element by element
        encoded_payload = bytearray()
        for item in rs_encoded:
            if isinstance(item, int):
                encoded_payload.append(item & 0xFF)
            else:
                encoded_payload.append(int(item) & 0xFF)
    
    # Create deterministic payload with signature
    final_payload, signature = self.create_deterministic_payload(encoded_payload)
    
    # Create metadata
    metadata = {
        'original_length': len(message_bytes),
        'encoded_length': len(encoded_payload),
        'final_length': len(final_payload),
        'signature_length': len(signature),
        'ecc_symbols': self.ecc_symbols,
        'header_structure': {
            'payload_len_bytes': 4,
            'sig_len_bytes': 4
        }
    }
    
    return final_payload, metadata

# PATCH: Fix the save_public_key method - cryptography API fix
def _patched_save_public_key(self, path: str):
    """Patched version - uses correct cryptography API method."""
    from cryptography.hazmat.primitives import serialization
    
    # Correct method: public_bytes() not public_key_bytes()
    pem = self.public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open(path, 'wb') as f:
        f.write(pem)

# Apply patches to fix the bytearray conversion bug and cryptography API
HybridMultiDomainEmbedderDet.create_deterministic_payload = _patched_create_deterministic_payload
HybridMultiDomainEmbedderDet.prepare_payload = _patched_prepare_payload
HybridMultiDomainEmbedderDet.save_public_key = _patched_save_public_key
print("✓ Applied comprehensive patches to fix bytearray conversion and cryptography API issues")

# Initialize embedder and verifier
# ULTRA-IMPERCEPTIBLE WATERMARKING: Optimized for maximum invisibility
# - Ultra-low alpha values (0.8-1.0) for minimal visual impact
# - Adaptive embedding adjusts strength based on local image characteristics
# - Optimized perceptual masking (0.3-1.5x) embeds conservatively
# - Minimal modification algorithms reduce changes to absolute minimum
# - Images should be NEARLY IDENTICAL to originals!

# Phase 1: Basic embedding/verification
embedder = HybridMultiDomainEmbedder(
    block_size_dct=8,
    block_size_dwt=16,
    ecc_symbols=20,
    alpha_dct=0.05,      # ULTRA-EXTREME low for maximum imperceptibility
    alpha_dwt=0.05,      # ULTRA-EXTREME low for maximum imperceptibility
    alpha_svd=0.1,     # ULTRA-EXTREME low for maximum imperceptibility
    redundancy=1,  # Reduced from 3 to minimize distortions
    use_adaptive=True,           # Enable adaptive embedding
    use_perceptual_masking=True  # Enable optimized perceptual masking
)

verifier = HybridMultiDomainVerifier(
    block_size_dct=8,
    block_size_dwt=16,
    ecc_symbols=20
)

# Phase 2: With tamper detection (using ultra-extreme-imperceptible settings)
embedder_det = HybridMultiDomainEmbedderDet(
    block_size_dct=8,
    block_size_dwt=16,
    ecc_symbols=20,
    alpha_dct=0.05,      # ULTRA-EXTREME low for maximum imperceptibility
    alpha_dwt=0.05,      # ULTRA-EXTREME low for maximum imperceptibility
    alpha_svd=0.1,     # ULTRA-EXTREME low for maximum imperceptibility
    redundancy=1,  # Reduced from 3 to minimize distortions
    use_adaptive=True,           # Enable adaptive embedding
    use_perceptual_masking=True  # Enable optimized perceptual masking
)

verifier_det = HybridMultiDomainVerifierDet(
    block_size_dct=8,
    block_size_dwt=16,
    ecc_symbols=20
)

print("✓ Models initialized successfully")
print("  Using MAXIMUM-IMPERCEPTIBLE watermarking settings:")
print("    - Color space: BGR (OpenCV standard, preserves RGB color)")
print("    - Embedding domain: DCT ONLY (most imperceptible, avoids distortion accumulation)")
print("    - Alpha DCT: 0.05 (99.4% reduction from original)")
print("    - Modification factor: 0.1x alpha (90% reduction in actual modifications)")
print("    - Redundancy: 1 (minimized to reduce distortions)")
print("    - Color preservation: Enabled (processes BGR channels separately)")
print("    - Adaptive embedding: Enabled (optimized)")
print("    - Perceptual masking: Enabled (conservative 0.3-1.5x)")
print("    - Single clipping: No double-clipping artifacts")
print("  ⭐ Watermarked images should be VISUALLY IDENTICAL to originals! ⭐")
print("  ⚠️  Note: Using DCT-only embedding for maximum imperceptibility.")
print("     Multi-domain embedding (DCT+DWT+SVD) is disabled to prevent distortion accumulation.")


## Step 4: Generate Watermarked Dataset

**Now processing each split (Train/Val/Test) separately to maintain data separation.**


In [ ]:
# Configuration: Choose which splits to process
PROCESS_TRAIN = True   # Process training set
PROCESS_VAL = True     # Process validation set  
PROCESS_TEST = True    # Process test set
# Set to False to skip a split (useful for testing with smaller datasets first)

message = "Copyright: Dataset 2024"
print(f"Generating watermarked dataset with message: '{message}'")
print("\nProcessing splits separately to maintain data separation:")

# Control which splits to process (set to True to process that split)
PROCESS_TRAIN = True   # Process training set
PROCESS_VAL = True     # Process validation set
PROCESS_TEST = True    # Process test set

all_metadata = {}

# Process Training Set
if PROCESS_TRAIN:
    print(f"\n{'='*60}")
    print("Processing TRAINING set...")
    print(f"{'='*60}")
    train_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "train"),
        output_dir=str(BASE_DIR / "dataset" / "train"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    train_metadata = train_gen.generate_watermarked_dataset(
        message=message,
        metadata_file="train_metadata.json"
    )
    all_metadata['train'] = train_metadata
    print(f"✓ Training: {len(train_metadata)} watermarked images")

# Process Validation Set
if PROCESS_VAL:
    print(f"\n{'='*60}")
    print("Processing VALIDATION set...")
    print(f"{'='*60}")
    val_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "val"),
        output_dir=str(BASE_DIR / "dataset" / "val"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    val_metadata = val_gen.generate_watermarked_dataset(
        message=message,
        metadata_file="val_metadata.json"
    )
    all_metadata['val'] = val_metadata
    print(f"✓ Validation: {len(val_metadata)} watermarked images")

# Process Test Set
if PROCESS_TEST:
    print(f"\n{'='*60}")
    print("Processing TEST set...")
    print(f"{'='*60}")
    test_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "test"),
        output_dir=str(BASE_DIR / "dataset" / "test"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    test_metadata = test_gen.generate_watermarked_dataset(
        message=message,
        metadata_file="test_metadata.json"
    )
    all_metadata['test'] = test_metadata
    print(f"✓ Test: {len(test_metadata)} watermarked images")

# Summary
total_watermarked = sum(len(meta) for meta in all_metadata.values())
print(f"\n{'='*60}")
print(f"SUMMARY: Generated {total_watermarked} total watermarked images")
print(f"{'='*60}")
for split_name, metadata in all_metadata.items():
    print(f"  {split_name.upper()}: {len(metadata)} images")


## Step 5: Generate Tampered/Attacked Dataset

This will generate attacked versions for each split (train/val/test) separately to maintain data separation.


In [ ]:
# Define attacks to apply
from training.attacks import AttackSimulator
import numpy as np

# Configure attacks
attacks = [
    ('jpeg_compression', {'quality': 75}),
    ('jpeg_compression', {'quality': 50}),
    ('crop', {'crop_percent': 0.9}),
    ('crop_corner', {'crop_percent': 0.1}),
    ('remove_region', {'x': 100, 'y': 100, 'width': 150, 'height': 150}),
    ('blur', {'kernel_size': 5}),
    ('blur', {'kernel_size': 7}),
    ('noise', {'noise_level': 0.02}),
    ('noise', {'noise_level': 0.05}),
    ('rotate', {'angle': 5}),
    ('rotate', {'angle': -10}),
    ('resize', {'scale': 0.8}),
    ('add_text', {'text': 'TAMPERED', 'position': (50, 50)}),
    ('add_text', {'text': 'WATERMARK', 'position': (100, 100)}),
    ('add_noise_patch', {'patch_size': (50, 50)}),
    ('draw_rectangle', {'pt1': (50, 50), 'pt2': (200, 200)}),
    ('draw_circle', {'center': (150, 150), 'radius': 50}),
    ('brightness', {'factor': 1.2}),
    ('contrast', {'factor': 1.3}),
]

print(f"Applying {len(attacks)} different attack types...")
print("Each watermarked image will get 3 random attacks applied.")
print("\nProcessing each split separately to maintain data separation...")

# Control which splits to process (set to True to process that split)
PROCESS_TRAIN = True   # Process training set
PROCESS_VAL = True     # Process validation set
PROCESS_TEST = True    # Process test set

all_tampered_metadata = {}

# Process Training Set
if PROCESS_TRAIN:
    print(f"\n{'='*60}")
    print("Generating tampered images for TRAINING set...")
    print(f"{'='*60}")
    train_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "train"),
        output_dir=str(BASE_DIR / "dataset" / "train"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    train_tampered = train_gen.generate_tampered_dataset(
        attacks=attacks,
        num_attacks_per_image=3
    )
    all_tampered_metadata['train'] = train_tampered
    print(f"✓ Training: Generated tampered images")

# Process Validation Set
if PROCESS_VAL:
    print(f"\n{'='*60}")
    print("Generating tampered images for VALIDATION set...")
    print(f"{'='*60}")
    val_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "val"),
        output_dir=str(BASE_DIR / "dataset" / "val"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    val_tampered = val_gen.generate_tampered_dataset(
        attacks=attacks,
        num_attacks_per_image=3
    )
    all_tampered_metadata['val'] = val_tampered
    print(f"✓ Validation: Generated tampered images")

# Process Test Set
if PROCESS_TEST:
    print(f"\n{'='*60}")
    print("Generating tampered images for TEST set...")
    print(f"{'='*60}")
    test_gen = DatasetGenerator(
        originals_dir=str(BASE_DIR / "dataset" / "test"),
        output_dir=str(BASE_DIR / "dataset" / "test"),
        embedder=embedder_det,
        verifier=verifier_det
    )
    test_tampered = test_gen.generate_tampered_dataset(
        attacks=attacks,
        num_attacks_per_image=3
    )
    all_tampered_metadata['test'] = test_tampered
    print(f"✓ Test: Generated tampered images")

print(f"\n{'='*60}")
print("✓ Tampered dataset generation complete for all splits!")
print(f"{'='*60}")
print("Data separation maintained - no leakage between train/val/test sets")


## Step 6: Verify Dataset


In [ ]:
# Verify watermarked and tampered images
print("Running verification on watermarked and tampered images...")

verification_results = dataset_gen.verify_dataset(verify_tampered=True)

# Print summary
wm_results = verification_results['watermarked']
tampered_results = verification_results['tampered']

wm_success = sum(1 for r in wm_results.values() if r.get('decode_success', False))
tamper_detected = sum(1 for r in tampered_results.values() 
                     if not r.get('decode_success', True) or not r.get('hash_valid', True))

print(f"\n{'='*60}")
print(f"VERIFICATION SUMMARY")
print(f"{'='*60}")
print(f"Watermarked images verified: {wm_success}/{len(wm_results)}")
print(f"Tampered images detected: {tamper_detected}/{len(tampered_results)}")
print(f"\nResults saved to: {BASE_DIR / 'dataset' / 'verification_results.json'}")


## Step 7: Run Test Harness (Robustness Testing)


In [ ]:
# Run comprehensive test harness
from training.test_harness_md_det import TestHarness
import json

# Initialize test harness
test_harness = TestHarness(
    embedder=embedder_det,
    verifier=verifier_det,
    output_dir=str(BASE_DIR / "outputs" / "eval_reports")
)

# Run tests on watermarked images
print("Running robustness tests...")
print("This will test various attacks and measure robustness metrics.")

# Get a sample of watermarked images for testing
watermarked_dir = BASE_DIR / "dataset" / "watermarked"
test_images = list(watermarked_dir.glob("*.png"))[:5]  # Test on first 5 images

if len(test_images) > 0:
    test_results = test_harness.run_tests(
        image_paths=[str(img) for img in test_images],
        message=message,
        num_attacks_per_image=5
    )
    
    print(f"\n✓ Test harness complete")
    print(f"Results saved to: {BASE_DIR / 'outputs' / 'eval_reports'}")
else:
    print("⚠ No watermarked images found for testing")


## Step 8: Train ML-Based Watermarking Models (Phase 3 - Optional)


In [ ]:
# Check if ML training is enabled
TRAIN_ML_MODELS = False  # Set to True to train ML models

if not TRAIN_ML_MODELS:
    print("ML model training is disabled. Set TRAIN_ML_MODELS = True to enable.")
    print("Note: ML training requires significant computational resources and time.")
else:
    import torch
    from torch.utils.data import DataLoader
    from training.train_ml_watermark import MLWatermarkTrainer, WatermarkDataset
    from models.ml_watermark_models import Encoder, Decoder, Discriminator
    
    # Check device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    # Hyperparameters
    message_bits = 100
    ecc_symbols = 20
    image_size = (256, 256)
    batch_size = 8
    num_epochs = 100
    
    # Create models
    encoder = Encoder(message_length=message_bits, image_channels=1)
    decoder = Decoder(message_length=message_bits, image_channels=1)
    discriminator = Discriminator(image_channels=1)
    
    # Create trainer
    trainer = MLWatermarkTrainer(
        encoder, decoder, discriminator,
        device=device,
        message_bits=message_bits,
        ecc_symbols=ecc_symbols,
        use_discriminator=True
    )
    
    # Create dataset from training split (maintains data separation)
    train_dataset = WatermarkDataset(
        image_dir=str(BASE_DIR / "dataset" / "train"),
        image_size=image_size
    )
    
    # Optional: Create validation dataset for monitoring during training
    val_dataset = WatermarkDataset(
        image_dir=str(BASE_DIR / "dataset" / "val"),
        image_size=image_size
    ) if (BASE_DIR / "dataset" / "val").exists() else None
    
    if len(train_dataset) == 0:
        print("⚠ No images found for training. Please add images to dataset/originals/")
    else:
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2 if device == 'cuda' else 0
        )
        
        print(f"Training on {len(train_dataset)} images")
        print(f"Batch size: {batch_size}, Epochs: {num_epochs}")
        
        # Train
        checkpoint_dir = BASE_DIR / "outputs" / "checkpoints"
        history = trainer.train(
            train_loader,
            num_epochs=num_epochs,
            save_dir=str(checkpoint_dir),
            save_interval=10
        )
        
        print("\n✓ ML model training complete!")
        print(f"Checkpoints saved to: {checkpoint_dir}")


In [ ]:
# Generate summary report
import json
from datetime import datetime

# Collect statistics
from training.utils_data import DatasetLoader

dataset_loader = DatasetLoader(str(BASE_DIR / "dataset"))
stats = dataset_loader.get_statistics()

# Load verification results if available
verification_file = BASE_DIR / "dataset" / "verification_results.json"
verification_data = {}
if verification_file.exists():
    with open(verification_file, 'r') as f:
        verification_data = json.load(f)

# Create summary report
report = {
    'timestamp': datetime.now().isoformat(),
    'dataset_statistics': stats,
    'verification_summary': {
        'watermarked_total': len(verification_data.get('watermarked', {})),
        'watermarked_success': sum(1 for r in verification_data.get('watermarked', {}).values() 
                                  if r.get('decode_success', False)),
        'tampered_total': len(verification_data.get('tampered', {})),
        'tampered_detected': sum(1 for r in verification_data.get('tampered', {}).values() 
                               if not r.get('decode_success', True) or not r.get('hash_valid', True)),
    },
    'directories': {
        'originals': str(BASE_DIR / "dataset" / "originals"),
        'watermarked': str(BASE_DIR / "dataset" / "watermarked"),
        'tampered': str(BASE_DIR / "dataset" / "tampered"),
        'outputs': str(BASE_DIR / "outputs"),
    }
}

# Save report
report_file = BASE_DIR / "outputs" / "summary_report.json"
with open(report_file, 'w') as f:
    json.dump(report, f, indent=2)

# Print summary
print("="*60)
print("SUMMARY REPORT")
print("="*60)
print(f"\nDataset Statistics:")
print(f"  Original images: {stats['originals']}")
print(f"  Watermarked images: {stats['watermarked']}")
print(f"  Tampered images: {stats['tampered']}")

if verification_data:
    print(f"\nVerification Results:")
    print(f"  Watermarked success rate: {report['verification_summary']['watermarked_success']}/{report['verification_summary']['watermarked_total']}")
    print(f"  Tampered detection rate: {report['verification_summary']['tampered_detected']}/{report['verification_summary']['tampered_total']}")

print(f"\nReport saved to: {report_file}")
print("\n✓ All processing complete!")


## Notes

- **Folder Structure**: The folders (watermarked, tampered, output, etc.) should be ready and can be empty initially. The code will create them if they don't exist and populate them with data.
- **Original Images**: Place your images in `dataset/originals/` before running Step 4.
- **ML Training**: Phase 3 ML model training is optional and can be enabled by setting `TRAIN_ML_MODELS = True` in Step 8.
- **Results**: All results are saved in the `outputs/` directory, including checkpoints, logs, and evaluation reports.
